In [1]:
# copy-paste from tutorials

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import RMSprop
from keras.datasets import mnist
from keras.utils import np_utils
from keras import initializers
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU


In [2]:
batch_size = 128
nb_classes = 10
epochs = 3

img_rows, img_cols = 28, 28         # input image dimensions
pool_size = (2, 2)                  # size of pooling area for max pooling
prob_drop_conv = 0.2                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.5              # drop probability for dropout @ fc layer


In [3]:
def init_weights(shape, name=None):
    return initializations.normal(shape, scale=0.01, name=name)

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print('X_train original shape:', X_train.shape)

#if K.backend.image_dim_ordering() == 'th':
#    # For Theano backend
#    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
#    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
#    input_shape = (1, img_rows, img_cols)
#else:
# For TensorFlow backend
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
    
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


11501568/11490434 [==============================] - 0s 0us/step
X_train original shape: (60000, 28, 28)
X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
# Convolutional model
model = Sequential()

# conv1 layer
model.add(Conv2D(32, (3,3), padding='same', activation='relu')) # input_shape=input_shape, init=init_weights))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(Dropout(prob_drop_conv))

# conv2 layer
model.add(Conv2D(64, (3,3), padding='same', activation='relu')) # , init=init_weights
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(Dropout(prob_drop_conv))

# conv3 layer
model.add(Conv2D(128, (3,3), padding='same', activation='relu')) # , init=init_weights
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(Flatten())
model.add(Dropout(prob_drop_conv))

# fc1 layer
#model.add(Dense(625, activation='relu')) # , init=init_weights
model.add(Dense(625, activation='linear')) # , init=init_weights
model.add(ELU())
model.add(Dropout(prob_drop_hidden))

# fc2 layer
model.add(Dense(10, activation='softmax')) # , init=init_weights

opt = RMSprop(lr=0.001, rho=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#model.summary()


2021-11-01 08:43:28.811721: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-01 08:43:28.868440: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-01 08:43:28.868739: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-01 08:43:28.869867: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
# Train
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, shuffle=True, verbose=1)


2021-11-01 08:43:30.014474: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/3


2021-11-01 08:43:31.461069: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204


469/469 [==============================] - 8s 8ms/step - loss: 0.2031 - accuracy: 0.9356
Epoch 2/3
469/469 [==============================] - 4s 8ms/step - loss: 0.0626 - accuracy: 0.9803
Epoch 3/3
469/469 [==============================] - 4s 8ms/step - loss: 0.0507 - accuracy: 0.9847


In [6]:
# Evaluate
evaluation = model.evaluate(X_test, Y_test, batch_size=256, verbose=1)
print('Summary: Loss over the test dataset: %.4f, Accuracy: %.4f' % (evaluation[0], evaluation[1]))


40/40 [==============================] - 0s 5ms/step - loss: 0.0325 - accuracy: 0.9891
Summary: Loss over the test dataset: 0.0325, Accuracy: 0.9891
